# 신경망으로 챗봇 만들기

이전 노트북에서는 코사인 유사성을 가진 챗봇을 만드는 방법을 배웠습니다. 이제는 신경망을 이용해 어떻게 만들 수 있는지 살펴보겠습니다!

훈련 데이터를 만들고 신경망을 훈련시킨 다음 훈련된 모델을 사용하여 챗봇을 만들 것입니다.

먼저, 필수 라이브러리를 설치할 것입니다. 라이브러리가 설치되지 않은 경우에만 아래 몇 개의 블록을 주석 해제하십시오.

In [1]:
#!pip install numpy scipy
#!pip install scikit-learn
# !pip install pillow
# !pip install h5py

In [2]:
# !pip install tensorflow

In [3]:
# !pip install tensorflow-gpu

In [4]:
#!pip install keras

# 1. 라이브러리 설치

우선 이 신경망 구동 챗봇에 필요한 라이브러리를 설치하겠습니다.
Keras는 백엔드에서 텐서플로우(다른 하위 레벨 기계 학습 라이브러리) 를 활용하는 기계 학습 라이브러리입니다. 이렇게 하면 우리의 목적을 위해 심층 신경망을 쉽게 배포할 수 있습니다.

In [5]:
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import Dense
 
from numpy import argmax
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

# 2. 입력 훈련 데이터

먼저 챗봇에 대한 다음 교육 데이터를 포함하겠습니다.:
1. X는 사용자가 입력할 수 있는 다양한 입력을 나타냅니다.
2. Y는 입력의 의도를 나타냅니다.

In [6]:
X = ['Hi',
     'Hello',
     'How are you?',
     'I am making',
     'making',
     'working',
     'studying',
     'see you later',
     'bye',
     'goodbye']

In [7]:
print(len(X))

10


In [8]:
Y = ['greeting',
     'greeting',
     'greeting',
     'busy',
     'busy',
     'busy',
     'busy',
     'bye',
     'bye',
     'bye']

In [9]:
print(len(Y))

10


비슷한 의도를 가진 여러 개의 다른 문장들이 있다는 것을 주목하십시오. 여기에서는 3개의 의도(greeting, busy, bye)만 있지만 프로젝트에 원하는 만큼 추가할 수 있습니다.

이것은 챗봇이 작동하는 방식입니다:
1. 입력 문장으로부터, 우리는 훈련된 AI 모델을 사용하여 의도를 확인할 것이다.
2. 각 의도에 대해, 우리는 준비 된 응답을 가지고있다.

예를 들어, 입력의 의도가 인사말에 대한 것임을 확인하면 챗봇에 '안녕하세요'또는 '어떻게 지내십니까?'와 같은 인사말로 응답하도록 요청할 수 있습니다.

우리는 기계 학습을 사용하여 입력 문장을 다른 의도로 분류 할 수있는 모델을 만들 것입니다. 
다음과 같이 만듭니다:

1. 문장과 그 의도에 대한 목록을 포함하고 있는 훈련 데이터(위의 X및  Y)를 작성한다.
2. 훈련 데이터를 사용하여 분류기를 훈련한다. 
3. 입력 문장을 벡터화하고 분류기를 사용하여 의도를 결정한다. 

# 3. 텍스트 처리

평소와 같이 텍스트 처리부터 시작합니다. 그 과정을 기억하십니까?

## 3.1 알파벳과 숫자가 아닌 문자 제거

In [10]:
def remove_non_alpha_numeric_characters(sentence):
    """
    문자열 하나를 입력받아 알파벳이 아닌 문자들을 제외하고 새로운 문자열로 구성하여 return 하는 함수
    """
    new_sentence = ''
    for alphabet in sentence:
        if alphabet.isalpha() or alphabet == ' ':
            new_sentence += alphabet
    return new_sentence

In [11]:
def preprocess_data(X):
    """
    배열의 모든 요소에 대해 전처리를 진행한다.
    1. 모든 요소를 소문자로 변경.
    2. 모든 요소에 remove_non_alpha_numeric_characters 함수를 이용해 알파벳이 아닌 문자열들을 제외.
    3. 모든 요소에서 공백을 제거한다.
    4. 반복되는 공백을 공백 한 칸으로 변경한다.
    """
    
    X = [data_point.lower() for data_point in X]
    X = [remove_non_alpha_numeric_characters(sentence) for sentence in X]
    X = [data_point.strip() for data_point in X]
    X = [re.sub(' +', ' ', data_point) for data_point in X]
    return X

In [12]:
X = preprocess_data(X) # 데이터 전처리

vocabulary = set() # BoW 생성을 위한 set 생성
for data_point in X:
    for word in data_point.split(' '):
        vocabulary.add(word)

vocabulary = list(vocabulary)

## 문서 벡터 생성

In [13]:
X_encoded = []

def encode_sentence(sentence):
    sentence = preprocess_data([sentence])[0]
    sentence_encoded = [0] * len(vocabulary)
    for i in range(len(vocabulary)):
        if vocabulary[i] in sentence.split(' '):
            sentence_encoded[i] = 1
    return sentence_encoded

X_encoded = [encode_sentence(sentence) for sentence in X]

In [14]:
classes = list(set(Y))

Y_encoded = []
for data_point in Y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)

# 4. 훈련 데이터 및 테스트 데이터 생성

In [15]:
X_train = X_encoded
y_train = Y_encoded
X_test = X_encoded
y_test = Y_encoded

훈련 및 테스트 데이터에 사용하는 데이터 출력 및 확인

In [16]:
print (y_test)

[[1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1]]


In [32]:
print((X_train))

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [18]:
y_train

[[1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1]]

y_train은 무엇을 의미합니까? 위에 표시된 배열을 이해합니까?

# 5. 모델 훈련

이제 훈련 데이터를 이용해 신경망을 훈련시키겠습니다.

In [19]:
model = Sequential()
model.add(Dense(units=64, activation='sigmoid',input_dim=len(X_train[0])))
model.add(Dense(units=len(y_train[0]), activation='softmax'))

model.compile(loss=categorical_crossentropy, optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))

model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=16)

Epoch 1/100



1/1 [==============================] - 0s 270ms/step - loss: 1.1449
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 1.1315
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 1.1160
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 1.1011
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0889
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 1.0806
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0762
Epoch 8/100
1/1 [==============================] - 0s 0s/step - loss: 1.0749
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 1.0754
Epoch 10/100
1/1 [==============================] - 0s 0s/step - loss: 1.0766
Epoch 11/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0773
Epoch 12/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0769
Epoch 13/100
1/1 [==============================] - 0s 2ms/step - loss: 1

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                960       
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 1155 (4.51 KB)
Trainable params: 1155 (4.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 예측 목록 표시

In [21]:
predictions = [argmax(pred) for pred in model.predict(np.array(X_test))]

1/1 [==============================] - 0s 60ms/step


# 모델 평가

이제 모델을 평가해 봅시다. 모델에 의한 예측과 테스트 데이터를 비교할 것입니다:

In [22]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == argmax(y_test[i]):
        correct += 1

print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 8
Total: 10


# 챗봇 테스트

이제 챗봇을 테스트해 보겠습니다! 문장을 입력한 다음 신경망에서 예측되는 클래스를 확인합니다:

In [23]:
# while True:
print ("Enter a sentence")
sentence = input()
prediction= model.predict(np.array([encode_sentence(sentence)]))
print (classes[argmax(prediction)])

Enter a sentence
1/1 [==============================] - 0s 20ms/step
busy


챗봇을 멈출 수 없다는 것을 알고 있습니까? 나중에 종료 명령을 추가해야 합니다(이전 노트를 참조하여 수행 방법을 확인하십시오.).

일단은 위의 중지 버튼(인터럽트 버튼)을 눌러서 챗봇을 중지하면 됩니다.

시도해 보세요. 정지 버튼을 누르고 상자에 뭔가를 입력해 보세요.

# 도전과제

우리는 성공적으로 대화 의도에 우리의 입력을 매핑하는 신경망을 사용했습니다. 
여러분의 과제는 대화 의도를 챗봇이 말하는 특정 응답과 연결하는 것입니다. 
예를 들어, 만약 대화의 목적이 '인사' 라면, 여러분의 챗봇도 인사말을 하도록 하세요!

In [24]:
# your code here

### 잘했습니다! 신경망으로 간단한 챗봇을 성공적으로 만들었습니다! 챗봇을 어떻게 개선할 수 있을까요?
다음과 같은 방법으로 챗봇을 개선할 수 있습니다:
- 더 많은 교육 데이터 추가
- 더 많은 의도 추가
- 특정 주제에 초점을 맞추고 해당 주제에 많은 훈련 데이터로 챗봇 교육

### 출처:
https://blog.eduonix.com/internet-of-things/simple-nlp-based-chatbot-python/